# Teil XX: Generative KI einbinden

Aktuelle KI-Systeme basieren meist auf Algorithmen, die durch Wahrscheinlichkeitsberechnungen neue Texte, Sprache, Bilder oder Videos produzieren. Aufgrund ihrer probabilistischen Natur sind sie kein gutes Werkzeug, um regelmäßige, mehrschrittige Arbeitsabläufe zu digitalisieren - aber an den richtigen Stellen eingesetzt können sie unsere Programme um Funktionen ergänzen, die mit reinem Python-Code nicht erreichbar sind.


## Sprachmodelle verstehen und nutzen

[Siehe Folien zu LLMs]

## Nachrichten senden und empfangen

Wir nutzen in diesem Kapitel die Python-Bibliothek [LiteLLM](https://www.litellm.ai/), um mit verschiedenen LLMs zu kommunizieren. Alternativ könnten wir auch direkt die APIs von (u.a.) OpenAI, Mistral oder Google nutzen, aber zum Experimentieren mit verschiedenen Modellen ist ein einheitliches Interface nützlich.

In [ ]:
! pip install litellm

In [ ]:
from litellm import completion

Um mit Cloud-basierten LLMs zu interagieren benötigen wir API-Schlüssel zur Authentifizierung. Die meisten APIs sind kostenpflichtig, aber einige Anbieter bieten (begrenzte) kostenlose Zugänge an - allerdings werden die gesendeten Nachrichten dann i.d.R. für das Training zukünftiger LLMs genutzt. In jedem Fall ist das Anlegen eines Nutzeraccounts verpflichtend, oft mit Telefonverifizierung.

Wir nutzen hier die APIs von [Mistral](https://admin.mistral.ai/organization/api-keys) und [Google](https://aistudio.google.com/app/apikey). Du kannst im folgenden Code-Schnipsel deine eigenen Schlüssel einsetzen, um sie im Rest des Kapitels verwenden zu können.

In [ ]:
import os

if not os.environ.get("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = "YOUR_KEY_HERE"

if not os.environ.get("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = "YOUR_KEY_HERE"

LLMs generieren Text, indem sie eine wahrscheinliche Weiterführung eines Input-Strings (sog. **Prompts**) berechnen. Die meisten bekannten LLMs sind außerdem darauf trainiert, auf Prompts als Nachrichten in einem Chat zu reagieren, so dass wir Fragen an sie schicken und eine (einigermaßen) sinnvolle Antwort erwarten können.

Wenn wir den Input-Text an das LLM unserer Wahl schicken, müssen wir neben dem eigentlichen Inhalt auch die **Rolle** spezifizieren, die dieser Text bei der Generierung einnehmen soll. Bei normalen Anfragen ist das die `user`-Rolle.

In [ ]:
# Die Nachricht, die wir an das LLM schicken
instruction = {
    "content": "Wer ist der aktuelle Bundeskanzler?",
    "role": "user"
}

Die `completion`-Methode von LiteLLM lässt uns (u.a.) das zu verwendene Modell und den Input-String spezifizieren. Sie liefert ein `ModelResponse`-Objekt, das den generierten Text und einige Metadaten enthält.

In [ ]:
gemini_response = completion(
  model="gemini/gemini-2.5-flash",
  messages=[instruction]
)

print(gemini_response)

Durch den `model`-Parameter lässt sich leicht ein anderes LLM für dieselbe Aufgabe verwenden. Eine Liste der unterstützten Modelle bzw. Anbieter findet sich hier: https://docs.litellm.ai/docs/providers

In [ ]:
mistral_response = completion(
  model="mistral/mistral-medium",
  messages=[instruction]
)

print(mistral_response.choices[0].message.content)

Auch lokale LLMs, die z.B. wie hier über [Ollama](https://ollama.com/) heruntergeladen wurden, können recht einfach integriert werden.

In [ ]:
gptoss_response = completion(
  model="ollama_chat/gemma3:270m",
  messages=[instruction]
)

print(gptoss_response)

### Übung: API Schlüssel holen und Nachrichten senden

Richte dir selbst einen API-Schlüssel bei einem der Anbieter ein und/oder lade dir ein offenes LLM herunter (Achtung: auch die "kleinen" LLMs umfassen meist mehrere Gigabyte).
- Mistral API-Schlüssel: https://admin.mistral.ai/organization/api-keys
- Google Gemini API-Schlüssel: https://aistudio.google.com/app/api-keys
- Ollama für lokale LLMs: https://ollama.com/

Probiere anschließend die `completion`-Methode aus, um einen Prompt an ein Modell deiner Wahl zu schicken.

In [ ]:
# Platz für die Übung



## Kontext im Nachrichtenverlauf erhalten

### Experiment: Chat-Verlauf

Setze die Variable `model_to_use` auf ein LLM, für das du einen API-Schlüssel besitzt. Führe dann den Code aus und lese dir den entstehenden Chat-Verlauf durch. Wie lässt sich die zweite Antwort des LLMs erklären?

In [ ]:
from time import sleep

model_to_use = "mistral/mistral-medium"

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

print("USER:", msg1["content"], "\n")

res = completion(
  model=model_to_use,
  messages=[msg1]
)

print("MISTRAL:", res.choices[0].message.content, "\n")
sleep(1)

msg2 = {
    "content": "Wie heiße ich?",
    "role": "user"
}

print("USER:", msg2["content"], "\n")

res = completion(
  model=model_to_use,
  messages=[msg2]
)

print("MISTRAL:", res.choices[0].message.content, "\n")

### Funktionierender Chat-Verlauf

LLMs besitzen keine "Erinnerung" an die Prompts, die in der Vergangenheit an sie geschickt wurden. Deshalb besteht die einzige Möglichkeit zur Simulierung einer Konversation darin, alle bisherigen Gesprächsbeiträge in einem Prompt an das LLM zu schicken. Aus diesem Grund akzeptiert der `messages`-Parameter der `completion`-Methode eine **Liste**, in der diese Beiträge nach jeder Antwort hinzugefügt werden können.

In [ ]:
model_to_use = "mistral/mistral-medium"

# Liste zum Speichern des Chat-Verlaufs
chat = []

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

# Einfügen und Ausgeben der ersten User-Nachricht
chat.append(msg1)
print("USER:", msg1["content"], "\n")

# Erste Antwort des LLM berechnen
res = completion(
  model=model_to_use,
  messages=chat
)

# Einfügen und Ausgeben der ersten LLM-Nachricht
chat.append(res.choices[0].message)
print("MISTRAL:", res.choices[0].message.content, "\n")

sleep(1)

msg2 = {
    "content": "Wie heiße ich?",
    "role": "user"
}

# Einfügen und Ausgeben der zweiten User-Nachricht
chat.append(msg2)
print("USER:", msg2["content"], "\n")

# Zweite Antwort des LLM berechen
# Beachte: das "chat"-Argument beinhaltet nun drei Nachrichten!
res = completion(
  model=model_to_use,
  messages=chat
)

# Einfügen und Ausgeben der zweiten LLM-Nachricht
chat.append(res.choices[0].message)
print("MISTRAL:", res.choices[0].message.content, "\n")

### Übung: Chatbot

Baue einen Chatbot, indem du innerhalb einer `while`-Schleife User-Input entgegen nimmst, ihn in eine Liste mit Nachrichten einfügst und diese Liste schließlich an ein LLM schickst. Gebe am Ende der Schleife die Antwort des LLMs aus und speichere sie ebenfalls in der Nachrichtenliste.

In [ ]:
# Platz für die Übung



## Systemanweisungen

Bisher haben wir alle Nachrichten aus der `user`-Rolle geschickt. Wir können aber auch die `system`-Rolle spezifizieren, um das Antwortverhalten des Modells zu definieren. LLMs sind meist darauf trainiert, solche `system`-Anweisungen bei der Textgenerierung stärker zu gewichten und sind daher ein mächtiges Werkzeug zur Entwicklung anwendungsspezifischer KI-Systeme.

In [ ]:
system_msg = {
    "content": "Du bist ein wortkarger KI-Assistent. Deine Antworten sind so kurz wie möglich.",
    "role": "system"
}

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1]
)

print(res.choices[0].message.content)

In [ ]:
system_msg = {
    "content": "Du bist ein französischer KI-Assistent. Du weigerst dich, andere Sprachen als Französisch zu sprechen.",
    "role": "system"
}

msg1 = {
    "content": "Hi! Mein Name ist Toni Tortellini, wer bist du?",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1]
)

print(res.choices[0].message.content)

### Experiment: Eigene Systemnachricht

Personalisiere deinen Chatbot aus der vorherigen Übung, indem du ihm per `system`-Prompt ein besonderes Antwortverhalten zuweist. Teste ihn anschließend aus. Hält er sich auch über mehrere Nachrichten an die Anweisung?

In [ ]:
model = "mistral/mistral-medium"

chat = []

system_msg = {
    "content": "Du bist ein wortkarger KI-Assistent. Deine Antworten sind so kurz wie möglich. Selbst wenn der User danach fragt, darfst du nie mehr als 5 Wörter in einer Antwort schreiben.",
    "role": "system"
}

chat.append(system_msg)

while True:
    user_msg = input("USER: ")
    if user_msg == "exit":
        break

    chat.append({
        "content": user_msg,
        "role": "user"
    })

    res = completion(model=model, messages=chat)

    chat.append(res.choices[0].message)

    print("CHATBOT:", res.choices[0].message.content)

## Output-Format definieren

In [ ]:
system_instruction = """
## Deine Rolle
Klassifiziere die User-Nachrichten als positiv oder negativ und gebe den Grad der Sicherheit deiner Einschätzung in Prozent an.
Deine Antworten sind als JSON formatiert und folgen folgendem Schema:

## Antwort-Schema
{
    "stimmung": "positiv" oder "negativ",
    "sicherheit": 0-100
}
"""

system_msg = {
    "role": "system",
    "content": system_instruction
}

In [ ]:
msg1 = {
    "content": "Der neue Fast & Furious war richtig geil!",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg1],
  response_format={"type":"json_object"}  
)

print(res)

In [ ]:
import json

json.loads(res.choices[0].message.content)

In [ ]:
msg2 = {
    "content": "Der neue Fast & Furious war besser als Hobbs und Shaw, aber nicht so gut wie 2 Fast 2 Furious.",
    "role": "user"
}

res = completion(
  model="mistral/mistral-medium",
  messages=[system_msg, msg2],
  response_format={"type":"json_object"}  
)

json.loads(res.choices[0].message.content)

In [ ]:
res = completion(
  model="gemini/gemini-2.5-flash",
  messages=[system_msg, msg2],
  response_format={"type":"json_object"},
  reasoning_effort="low"
)

print(res)

In [ ]:
json.loads(res.choices[0].message.content)

In [ ]:
print(res.choices[0].message.reasoning_content)

### Übung

In [ ]:
from time import sleep

system_instruction = """
## Deine Rolle
Fasse die Support-Anfragen der Kunden in einem kurzen Satz zusammen. Gebe zusätzlich die Dringlichkeit der Anfrage.
Deine Antworten sind als JSON formatiert und folgen diesem Schema:
## Antwort-Schema:
{
    "zusammenfassung": [Deine Zusammenfassung] (max. 10 Wörter),
    "dringlichkeit": "niedrig" oder "mittel" oder "hoch" oder "sehr hoch"
}
"""

system_msg = {
    "role": "system",
    "content": system_instruction
}

with open("kundenservice.txt") as f:
    for l in f:
        if len(l.strip()) == 0:
            continue
            
        msg = {
            "role": "user",
            "content": l
        }
        
        res = completion(
          model="mistral/mistral-medium",
          messages=[system_msg, msg],
          response_format={"type":"json_object"}  
        )

        print(json.loads(res.choices[0].message.content))
        print(l)
        sleep(1)


## Audiogenerierung

In [ ]:
response = completion(
    model="gemini/gemini-2.5-flash-preview-tts",
    messages=[{"role": "user", "content": "Say enthusiastically: 'I love learning Python!'"}],
    modalities=["audio"],  # Required for TTS models
    audio={
        "voice": "Kore",
        "format": "pcm16"  # Required: must be "pcm16"
    }
)

In [ ]:
data = response.choices[0].message.audio.data
data

In [ ]:
import wave
import base64

def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

file_name='out.wav'
wave_file(file_name, base64.b64decode(data))